### 1、Import tensorflow and use tensorflow to define a constant.

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

ModuleNotFoundError: No module named 'tensorflow'

### 2、Use tensorflow to print 'Welcome to Huawei'.

In [138]:
#1.2.1	Setting Encoding Declarations
# -*- coding: utf-8 -*-
#1.2.2	Importing a Module
# import tensorflow as tf
#1.2.3	Defining Variables
hello = tf.constant('Welcome to Huawei')  #Define a constant.
#1.2.4	Creating a Session
sess = tf.Session()                            #Create a session.
print (sess.run(hello))                        #Run the command session.run to get the result.
#1.2.5	Closing the Session
sess.close()    

b'Welcome to Huawei'


### 3、Use tensorflow to define two numeric constants,then calculate them and print the result.

In [139]:
#2.2	Experimental Process
#2.2.1	Setting Encoding Declarations
# -*- coding: utf-8 -*-
#2.2.2	Importing a Module
# import tensorflow as tf
#2.2.3	Defining Constants
a = tf.constant(3)                     #Defining constant 3.
b = tf.constant(4)                     #Defining constant 4.
#2.2.4	Creating a Session
with tf.Session() as sess:           #Create a session.
    print("Add:  %i" % sess.run(a+b))     #Calculate and output the sum of the two constants

    print ("Multiply:  %i" % sess.run(a*b))     #Calculate and output the product of the two constants.

Add:  7
Multiply:  12


### 4、Randomly generate two matrices using tensorflow, calculate and print the result of their multiplication.

In [140]:
sess = tf.InteractiveSession()
w1 = tf.Variable(tf.random_normal([2,3],mean=1.0, stddev=1.0))
w2 = tf.Variable(tf.random_normal([3,1],mean=1.0, stddev=1.0))
# x = tf.constant([[0.7, 0.9]])
tf.global_variables_initializer().run()
# a = tf.matmul(x ,w1)
# y = tf.matmul(a, w2)
mul = tf.matmul(w1, w2)
# print(y.eval())
print(sess.run(mul))
sess.close() 

[[3.3922825]
 [3.919043 ]]


In [141]:
!pip install -q git+https://github.com/tensorflow/examples.git

  ERROR: Error [WinError 2] Не удается найти указанный файл while executing command git clone -q https://github.com/tensorflow/examples.git 'C:\Users\Dmitry\AppData\Local\Temp\pip-req-build-5k56gaif'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


### 5、Try to use your own handwriting number to do image recognition

In [162]:
# import numpy as np
# # import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# from tensorflow.examples.tutorials.mnist import input_data
# import time
# import cv2
# import matplotlib.pyplot as plt
# mnist = input_data.read_data_sets("data\\MNIST_data\\", one_hot=True)

import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# from tensorflow.examples.tutorials as input_data
import tensorflow_dataset
# import tensorflow_datasets
import time
import cv2
import matplotlib.pyplot as plt

# mnist = input_data.read_data_sets("Experiments\More\data\\MNIST_data\\", one_hot=True)
# mnist = input_data.read_data_sets("data\\MNIST_data\\", one_hot=True)
mnist = tensorflow_datasets.load('mnist')

def make_label(label_num):
    label = np.zeros((1,10),dtype='float32')
    label[:,label_num] = 1.0
    return label

label_test = make_label(6)
# the path of image
img_path = "data\\self_data\\6.jpg"
img_file = cv2.imread(img_path)
img_file = cv2.cvtColor(img_file, cv2.COLOR_RGB2GRAY)
img_file = 255 - img_file
for row in range(img_file.shape[0]):
    for col in range(img_file.shape[1]):
        if img_file[row][col] < 120.0:
            img_file[row][col] = 0
plt.imshow(img_file,'gray')
plt.show()
data_test = img_file / 255
data_test = np.float32(img_file.reshape(1, 28*28))
data_test.shape

ModuleNotFoundError: No module named 'tensorflow_dataset'

In [160]:
input_num = 784
h1_num = 512
epochs = 3000
batch_num = 1000
lr = 0.09
W1 = tf.Variable(tf.truncated_normal([input_num, h1_num],stddev=0.1))
W2 = tf.Variable(tf.zeros([h1_num,10]))
B1 = tf.Variable(tf.zeros([h1_num]))
B2 = tf.Variable(tf.zeros([10]))
x = tf.placeholder(tf.float32, [None, input_num])
keep_prob = tf.placeholder(tf.float32)
label = tf.placeholder(tf.float32, [None,10])

hidden1 = tf.nn.relu(tf.matmul(x,W1) + B1)
hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
y = tf.nn.softmax(tf.matmul(hidden1_drop, W2) + B2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(label * tf.log(y), reduction_indices=[1]))
train_step = tf.train.AdagradOptimizer(lr).minimize(cross_entropy)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [161]:
with tf.Session() as sess:
    start = time.time()
    tf.global_variables_initializer().run()
    for epoch in range(epochs):
        batch_xs, batch_ys = mnist.train.next_batch(batch_num)
        _,cost = sess.run([train_step, cross_entropy], feed_dict={x: batch_xs, label: batch_ys, keep_prob:0.75})
        if epoch % 100 == 0:
            print("epoch: {}, loss: {:.2f}".format(epoch, cost))
        
    stop = time.time()
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(label,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    predict_acc = sess.run(accuracy, feed_dict={x: mnist.test.images, label: mnist.test.labels, keep_prob:1.0})
    print("The accuracy of the test data：{:.2f}%".format(predict_acc * 100))
    print("The total time：{:.2f}s".format(stop-start))
    test_acc,test_value = sess.run([accuracy,y], feed_dict={x:data_test, label:label_test, keep_prob:1.0})
    print("The accuracy of you:{:.2f}%".format(test_acc * 100))
    
print("The prediction by AI is{}".format(list(test_value[0]).index(test_value[0].max())))

AttributeError: module 'tensorflow.python.keras.api._v1.keras.datasets.mnist' has no attribute 'train'